In [1]:

import sys, os
from pyspark.sql.functions import col, explode # lit, cast  
from pyspark.sql.types import DateType, TimestampType   # sha2,concat_ws,trim, lit

# Insert the parent directory (one level up) onto Python’s module search path
sys.path.insert(0, os.path.abspath(".."))

from dev_spark_session import DevSparkSession 
from get_stock_data import GetStockData
from scd_type2_handler import SCDType2Handler
from delta.tables import DeltaTable



### Initiate Spark

In [2]:
stockdata = GetStockData() 
spark = DevSparkSession().spark

ERROR StatusLogger Reconfiguration failed: No configuration found for '5ffd2b27' at 'null' in 'null'
ERROR StatusLogger Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'


:: loading settings :: url = jar:file:/Users/PC/Desktop/VS%20Code%20Repositories/azure-stock-market/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/PC/.ivy2/cache
The jars for the packages stored in: /Users/PC/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a242f45d-e6ec-488a-be59-40eade66ed27;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 90ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   

### Get Data from API

In [ ]:
json_data  = stockdata.get_historical_stock_data()


df = spark.createDataFrame(json_data) 

df = df.withColumn("symbol", col("meta").symbol).filter( col("status") == "ok" ).drop(df.status)

# df.show(truncate = False)


df.write \
    .format("parquet") \
    .mode("overwrite") \
    .partitionBy( "symbol" ) \
    .save("/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze")



### Bronze -> Silver

In [ ]:


# Paths for Bronze and Silver data
bronze_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Bronze"  # Bronze data stored in Parquet or Delta
silver_path = "/Users/PC/Desktop/VS Code Repositories/azure-stock-market/Azure storage/Silver/delta-table"  # Target location for Silver Delta table



### Tranformt Bronze -> Silver

df_bronze = spark.read.load(bronze_path).drop("symbol")

df_bronze = df_bronze.withColumn( "values", explode("values") )

df_bronze = df_bronze.select(
    col("meta").getItem("symbol").alias("Symbol"),
    col("meta").getItem("exchange").alias("ExchangeName"),
    col("meta").getItem("currency").alias("Currency"),
    col("meta").getItem("type").alias("Type"),
    col("meta").getItem("exchange_timezone").alias("ExchangeTimeZone"),
    col("values").getItem("volume").alias("Volume"),
    col("values").getItem("high").alias("High"),
    col("values").getItem("low").alias("Low"),
    col("values").getItem("close").alias("Close"),
    col("values").getItem("open").alias("Open"),
    col("values").getItem("datetime").alias("Date"))

df_bronze = df_bronze.dropDuplicates()


# df_bronze.show()


In [ ]:
parameters = {
        "businessColumns" : "Symbol,ExchangeName,Currency,Date",
        "typeIColumns" : "Symbol"
        }

scd2Handler =  SCDType2Handler(parameters)
scd2Handler.refresh_timestamp()
add_audit_columns =  scd2Handler.add_audit_columns

df_bronze = df_bronze.transform(add_audit_columns)

# df_bronze_audit = df_bronze.transform(scd2Handler.add_audit_columns)
# df_bronze_audit.show()
df_bronze.show(truncate=False)
deltaTable = DeltaTable.forPath(spark, silver_path)
scd2Handler.delta_merge_typeII(deltaTable, df_bronze)

In [ ]:
df_history = spark.sql(f"""
    DESCRIBE HISTORY delta.`{silver_path}` 
    LIMIT 1
""")

df_history = df_history.select("operationMetrics").head()[0]


audit =   { "outputrows" :  df_history["numOutputRows"],
            "inserted": df_history["numTargetRowsInserted"],
            "updated": df_history["numTargetRowsUpdated"],
            "deleted": df_history["numTargetRowsDeleted"] }
        

audit = { key : value for key, value  in audit.items() if value != '0' }

audit

In [ ]:
spark.sql (f""" create table if not exists silve 
           using delta
           location '{silver_path}'""")

In [ ]:
spark.conf.get("spark.sql.catalogImplementation")

In [ ]:
%load_ext sql
